importing json module and reading data as well as checking type, format of data

In [1]:
import json
from time import time
import random
tag_count={}
now=time()
f = open('penn-data.json')
data = json.load(f)
#print(data)
#random.shuffle(data)
for point in data:
    tags=point[1]
    for tag in tags:
        if tag not in tag_count:
            tag_count[tag]=1
        else:
            tag_count[tag]+=1
freq=0
most_used_tag=''
total=0
for tag,count in tag_count.items():
    total+=count
    if count>freq:
        freq=count
        most_used_tag=tag
        
print('Total number of words is:',total )        
print('Most used tag is: ',most_used_tag)
print(f'It is used for {freq} times ')
print("total number of tags in dataset is: ",len(tag_count))

Total number of words is: 81147
Most used tag is:  NN
It is used for 12721 times 
total number of tags in dataset is:  41


In [2]:
train_nos=round(0.8*len(data))
print('Total number of examples: ',len(data))
print('No. of training examples: ',train_nos)
test_nos=len(data)-train_nos
print('No. of testing examples: ',test_nos)

Total number of examples:  3914
No. of training examples:  3131
No. of testing examples:  783


In [3]:
word_dict={'<start>':0}
tag_dict={'<start>':0}
word_count=1
tag_count=1
seq_file = open("seq.txt","w")
tag_file=open("tag.txt","w")
for tag_seq in data:
    seq=tag_seq[0].split(' ')
    tag=tag_seq[1]
    for i in range(len(seq)):
        seq[i]=seq[i].lower()
        if "'" in seq[i]:
            seq[i]=seq[i].replace("'",'',100)
        if '.' in seq[i]:
            seq[i]=seq[i].replace('.','',100)
        if ',' in seq[i]:
            seq[i]=seq[i].replace(',','',100)
        if ',' in seq[i]:
            seq[i]=seq[i].replace('"','',100)
        if '``' in seq[i]:
            seq[i]=seq[i].replace('``','',100)
            
    seq=['<start>']+seq+['<end>']
    out_seq=' '
    out_tag=' '
    out_seq=out_seq.join(seq)
    
    tag=['<start>']+tag+['<end>']
    out_tag=out_tag.join(tag)
    seq_file.write(out_seq)
    seq_file.write('\n')
    tag_file.write(out_tag)
    tag_file.write('\n')


seq_file.close()
tag_file.close()

In [4]:
tag_file=open("tag.txt","r")
seq_file = open("seq.txt","r")
train_seq=open("seq_train.txt","w")
train_tag=open("tag_train.txt","w")
seq_file.seek(0)
tag_file.seek(0)
word_dict={'<start>':0}
tag_dict={'<start>':0}
word_count=1
tag_count=1
for i in range(train_nos):
    temp=seq_file.readline().split(' ')
    out=' '
    for word in temp:
        if word not in word_dict:
            word_dict[word]=word_count
            word_count+=1
    x=[str(word_dict[word]) for word in temp]
    out=out.join(x)
    train_seq.write(out)
    train_seq.write('\n')
    temp=tag_file.readline().split(' ')
    out=' '
    for tag in temp:
        if tag not in tag_dict:
            tag_dict[tag]=tag_count
            tag_count+=1
    x=[str(tag_dict[tag]) for tag in temp]
    out=out.join(x)
    train_tag.write(out)
    train_tag.write('\n')
test_tag=open('test_tag.txt','w')
test_seq=open('test_seq.txt','w')
for j in range(train_nos,len(data)):
    temp_seq=seq_file.readline()
    temp_tag=tag_file.readline()
    test_tag.write(temp_tag)
    #test_tag.write('\n')
    test_seq.write(temp_seq)
    #test_seq.write('\n')
train_tag.close()
train_seq.close()
tag_file.close()
seq_file.close()
test_tag.close()
test_seq.close()

In [5]:
tag_file=open('tag.txt','r')
for i in range(len(data)):
    temp=tag_file.readline().split(' ')
    for tag in temp:
            if tag not in tag_dict:
                tag_dict[tag]=tag_count
                tag_count+=1
tag_file.close()

In [6]:
import numpy as np
emit=np.zeros([word_count,tag_count])
emit_con=np.zeros([word_count,tag_count,tag_count])
trans=np.zeros([tag_count,tag_count])
trans_tri=np.zeros([tag_count,tag_count,tag_count])

In [7]:
train_seq1=open('seq_train.txt','r')
train_seq=train_seq1.readlines()
train_tag1=open('tag_train.txt','r')
train_tag=train_tag1.readlines()
for i in range(len(train_seq)):
    seq=[int(j) for j in train_seq[i].split()]
    tag=[int(j) for j in train_tag[i].split()]
    for j in range(len(seq)):
        emit[seq[j]][tag[j]]+=1
    for j in range(1,len(seq)):
        trans[tag[j]][tag[j-1]]+=1
        
    for j in range(2,len(seq)):
        emit_con[seq[j]][tag[j-1]][tag[j-2]]+=1
        trans_tri[tag[j]][tag[j-1]][tag[j-2]]+=1
train_seq1.close()
train_tag1.close()
#print(trans_tri)

In [8]:
emit/=np.sum(emit,axis=0)
trans/=np.sum(trans,axis=0) 
x,y,z=emit_con.shape
for i in range(x):
    emit_con[i]/=np.sum(emit_con[i],axis=0)
x,y,z=trans_tri.shape
for i in range(x):
    trans_tri[i]/=np.sum(trans_tri[i],axis=0)

<ipython-input-8-8da94e501538>:2: RuntimeWarning: invalid value encountered in true_divide
  trans/=np.sum(trans,axis=0)
<ipython-input-8-8da94e501538>:5: RuntimeWarning: invalid value encountered in true_divide
  emit_con[i]/=np.sum(emit_con[i],axis=0)
<ipython-input-8-8da94e501538>:8: RuntimeWarning: invalid value encountered in true_divide
  trans_tri[i]/=np.sum(trans_tri[i],axis=0)


In [9]:
test_seq=open("test_seq.txt",'r')
test_seq.seek(0)
missing=0
for line in test_seq:
    for word in line.split(' '):
        if word not in word_dict:
            missing+=1
print('Number of missing words is: ',missing)
test_seq.close()

Number of missing words is:  1820


In [10]:
test_seq=open("test_seq.txt",'r')
test_seq.seek(0)
print(test_seq.readline())
test_seq.close()

<start> the latest 10-year notes were quoted at 100 22/32 to yield 788% compared with 100 16/32 to yield 790% <end>



In [11]:
start_tag_num=word_dict['<start>']
end_tag_num=tag_dict['<end>\n']

In [12]:
class node:
    
    def __init__(self,tag_num,word,index):
        self.tag_num=tag_num
        self.word=word
        self.emit_prob=0
        self.bi=None
        self.tri=None
        self.prob=0
        self.tri=None
        self.index=index
        
        if self.word not in word_dict:
            self.emit_prob=1
            self.tag_num=tag_dict[most_used_tag]
        else:
            self.emit_prob=emit[word_dict[word]][self.tag_num]
            
class timestep:
    def __init__(self,word,index):
        self.word=word
        self.index=index
        self.node_list=[]
        self.probable_nodes=[]#list to store top 3 nodes 
        if word in word_dict:
            for i in range(tag_count):
                if emit[word_dict[word]][i]>0:
                    obj=node(tag_num=i,word=self.word,index=self.index)#tag_num,word,index.
                    self.node_list.append(obj)
        else:
            obj=node(tag_num=3,word=self.word,index=self.index)#tag_num,word,index.
            self.node_list.append(obj)
            
        #print(len(self.node_list),self.word)

class timeline:
    def __init__(self,seq):
        self.seq=seq
        self.tag_count=tag_count
        self.timesteps=[]
        for i in range(len(self.seq)):
            self.timesteps.append(timestep(word=seq[i],index=i))#word, index
        #print(len(self.timesteps))
            
    def trigram(self):
        #index0,1,2
        node1list=self.timesteps[1].node_list
        for node1 in node1list:
            node1.prob=node1.emit_prob*trans[node1.tag_num][tag_dict['<start>']]
            node1.tri=self.timesteps[0].node_list[0]
            #print(node1.prob)
        node2list=self.timesteps[2].node_list
        for node2 in node2list:
            node1list=self.timesteps[1].node_list
            temp=0
            temp_obj=None
            for node1 in node1list:
                resultant_prob=node2.emit_prob*trans[node2.tag_num][node1.tag_num]*node1.prob
                if resultant_prob==0:
                    resultant_prob=0.000000000000000001
                if resultant_prob>temp:
                    temp_obj=node1
                    temp=resultant_prob
            node2.tri=temp_obj
            node2.prob=temp
            #print(node2.prob)
                    
        for i in range(3,len(self.timesteps)):
            node2list=self.timesteps[i].node_list#current
            temp=0
            temp_obj=None
            temp_prob=0
            for node2 in node2list:
                #print(node2.emit_prob)
                node1list=self.timesteps[i-1].node_list#previous
                for node1 in node1list:
                    acc_prob=node1.prob
                    #print(acc_prob)
                    node0list=self.timesteps[i-2].node_list#previous to previous
                    for node0 in node0list:
                        if node2.word in word_dict:
                            emit_prob=emit_con[word_dict[word]][node1.tag_num][node0.tag_num]
                        else:
                            emit_prob=1
                        #print('emit_prob',' ' ,emit_prob)
                        #print(trans_tri[node2.tag_num][node1.tag_num][node0.tag_num],)
                        resultant_prob=acc_prob*emit_prob*trans_tri[node2.tag_num][node1.tag_num][node0.tag_num]
                        #print(resultant_prob)
                        if resultant_prob==0:
                            resultant_prob=0.00000000000000000001
                        if resultant_prob>temp_prob:
                            temp_obj=node1
                            temp_prob=resultant_prob
                            #print(node2.index)
                #print()
                node2.tri=temp_obj
                
                node2.prob=temp_prob
                
                
        last_ts=self.timesteps[-1].node_list#last character 
        temp=0
        out_node=None
        max_prob=[]
        for node1 in last_ts:
            #print(node1.word)
            if node1.prob>temp:
                temp=node1.prob
                out_node=node1
                #print(temp)
        
        out_tags=[]
        current_node=out_node
        while current_node:
            out_tags.append(current_node.tag_num)
            #print(current_node.index)
            current_node=current_node.tri
        out_tags=out_tags[::-1]    
        return out_tags    
    
    def trigram_3(self):
        #using probable node list in timestep class to store top 3 nodes
        node1list=self.timesteps[1].node_list
        for node1 in node1list:
            node1.prob=node1.emit_prob*trans[node1.tag_num][tag_dict['<start>']]
            node1.tri=self.timesteps[0].node_list[0]
        node1list.sort(key=lambda obj: obj.prob)#sorting list to get top three nodes
        if len(node1list)>=3:
            self.timesteps[1].probable_nodes=node1list[0:3] 
        else:
            self.timesteps[1].probable_nodes=node1list
        node2list=self.timesteps[2].node_list
        for node2 in node2list:
            node1list=self.timesteps[1].node_list
            temp=0
            temp_obj=None
            for node1 in node1list:
                resultant_prob=node2.emit_prob*trans[node2.tag_num][node1.tag_num]*node1.prob
                if resultant_prob==0:
                    resultant_prob=0.000000000000000001
                if resultant_prob>temp:
                    temp_obj=node1
                    temp=resultant_prob
            node2.tri=temp_obj
            node2.prob=temp
        node2list.sort(key=lambda obj: obj.prob)#sorting list to get top three nodes
        if len(node2list)>=3:
            self.timesteps[2].probable_nodes=node2list[0:3] 
        else:
            self.timesteps[2].probable_nodes=node2list
                    
        for i in range(3,len(self.timesteps)):
            node2list=self.timesteps[i].node_list
            temp=0
            temp_obj=None
            temp_prob=0
            for node2 in node2list:
                node1list=self.timesteps[i-1].node_list
                for node1 in node1list:
                    acc_prob=node1.prob
                    node0list=self.timesteps[i-2].node_list
                    for node0 in node0list:
                        if node2.word in word_dict:
                            emit_prob=emit_con[word_dict[word]][node1.tag_num][node0.tag_num]
                        else:
                            emit_prob=1
                        resultant_prob=acc_prob*emit_prob*trans_tri[node2.tag_num][node1.tag_num][node0.tag_num]
        
                        if resultant_prob==0:
                            resultant_prob=0.00000000000000000001
                        if resultant_prob>temp_prob:
                            temp_obj=node1
                            temp_prob=resultant_prob
            
                node2.tri=temp_obj
                node2.prob=temp_prob
        node2list.sort(key=lambda obj: obj.prob)#sorting list to get top three nodes
        if len(node2list)>=3:
            self.timesteps[i].probable_nodes=node2list[0:3] 
        else:
            self.timesteps[i].probable_nodes=node2list
                
        last_ts=self.timesteps[-1].node_list
        temp=0
        out_node=None
        max_prob=[]
        for node1 in last_ts:
            if node1.prob>temp:
                temp=node1.prob
                out_node=node1
        
        out_tags=[]
        current_node=out_node
        while current_node:
            out_tags.append(current_node.tag_num)
            current_node=current_node.tri
        out_tags=out_tags[::-1]    
        return out_tags        
                    
                        
        


In [19]:
num_to_tag={val:key for key,val in tag_dict.items()}
c='<start> that former sri lanka skipper and ace batsman aravinda se silva is a man of few words was very much evident on Wednesday when the legendary batsman who has always let his bat talk struggled to answer a barrage of questions at a function to_f promote <end>'
c=c.split()

t=timeline(c)
out1=t.trigram_3()#using top3 nodes at eah time step
out2=t.trigram()#using traditional viterbi
out1=[num_to_tag[i] for i in out1]
out2=[num_to_tag[i] for i in out2]
print(out1[1::])
print()
print(out2[1::])

['IN', 'JJ', 'NN', 'NN', 'NN', 'IN', 'NN', 'NN', 'NN', 'NN', 'NN', 'VBZ', 'JJ', 'NN', 'IN', 'JJ', 'NNS', 'VBD', 'RB', 'JJ', 'JJ', 'RB', 'NN', 'WRB', 'NNP', 'NN', 'NN', 'WP', 'VBZ', 'RB', 'VBP', 'PRP$', 'NNP', 'VB', 'VBD', 'JJ', 'VB', 'LS', 'NN', 'IN', 'NNS', 'IN', 'LS', 'NN', 'NN', 'VBP', 'NN']

['IN', 'JJ', 'NN', 'NN', 'NN', 'IN', 'NN', 'NN', 'NN', 'NN', 'NN', 'VBZ', 'JJ', 'NN', 'IN', 'JJ', 'NNS', 'VBD', 'RB', 'JJ', 'JJ', 'RB', 'NN', 'WRB', 'NNP', 'NN', 'NN', 'WP', 'VBZ', 'RB', 'VBP', 'PRP$', 'NNP', 'VB', 'VBD', 'JJ', 'VB', 'LS', 'NN', 'IN', 'NNS', 'IN', 'LS', 'NN', 'NN', 'VBP', 'NN']


In [20]:
c=c[1::]
out1=out1[1::]
for i in range(len(out1[1::])):
    print(c[i],out1[i])


that IN
former JJ
sri NN
lanka NN
skipper NN
and IN
ace NN
batsman NN
aravinda NN
se NN
silva NN
is VBZ
a JJ
man NN
of IN
few JJ
words NNS
was VBD
very RB
much JJ
evident JJ
on RB
Wednesday NN
when WRB
the NNP
legendary NN
batsman NN
who WP
has VBZ
always RB
let VBP
his PRP$
bat NNP
talk VB
struggled VBD
to JJ
answer VB
a LS
barrage NN
of IN
questions NNS
at IN
a LS
function NN
to_f NN
promote VBP
